In [ ]:
# Need to download tkHyperLinkManager.ipynb before running

In [ ]:
import requests
import json
from tkinter import *
from tkinter import messagebox
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import webbrowser
import import_ipynb
from tkHyperLinkManager import HyperlinkManager
from functools import partial
import datetime

# Link to weather API:
# https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/
api_key = "4P29EVE6MTFCAAH7CEKL22XLZ"

root = Tk()
root.title("Hotels & Weather App")

# Creating string variables for location, date1, and date2
# and setting them to empty strings
location = StringVar()
location.set("")

date1 = StringVar()
date1.set("")

date2 = StringVar()
date2.set("")

# List of numbers for the dropdown menu
number_list = [1, 2, 3, 4, 5, 6]

# Function that retrieves weather data
def get_weather(location, date1, date2):
    # Checking for valid inputs
    if not check_valid_input(location, date1, date2):
        return
    
    # Deleting any text currently in the weather_text widget
    weather_text.delete("1.0", "end")
    
    # Getting just the months and days that the user inputs
    # so we can get weather data for those days in past years
    start_month = date1[5:7]
    start_day = date1[8:10]

    end_month = date2[5:7]
    end_day = date2[8:10]

    # Initializing the total of all of the high temperatures 
    # for each day in the time period 
    # so we can get the average high temp
    total_tempmax = 0
    # Doing the same for low temp
    total_tempmin = 0
    # And for humidity
    total_humidity = 0

    #Initializing number of days in the time period provided by the user
    number_of_days = 0
    # Setting the number of years we want to retrieve past weather data for
    number_of_years = 5

    # Loop that allows us to retreive weather data for 
    # each day in the time period over the past five years
    for i in range(number_of_years):
        start_year = int(date1[:4]) - i - 1
        end_year = int(date2[:4]) - i - 1
        new_date1 = f"{start_year}-{start_month}-{start_day}"
        new_date2 = f"{end_year}-{end_month}-{end_day}"

        # Making sure location is valid and then making a call to the API 
        weather_json = None
        try:
            weather = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{new_date1}/{new_date2}?key={api_key}&include=days")
            weather_json = weather.json()
        except:
            messagebox.showerror("error", "invalid location")
            return

        # Loop that retrieves the high temp for each day,
        # adds them together, and divides by the number of days to get the average
        # Also does the same for the low temp and humidity
        days = weather_json["days"]
        for day in days:
            tempmax = day["tempmax"]
            tempmin = day["tempmin"]
            humidity = day["humidity"]

            number_of_days += 1

            total_tempmax += tempmax
            total_tempmin += tempmin
            total_humidity += humidity

            average_tempmax = int(total_tempmax / number_of_days)
            average_tempmin = int(total_tempmin / number_of_days)
            average_humidity = int(total_humidity / number_of_days)
        
    # Adding this text into the weather_text box
    text = f"Average low temp: {average_tempmin}°F\nAverage high temp: {average_tempmax}°F\nAverage humidity: {average_humidity}%\n"
    weather_text.insert(END, text)
    
# Function that scrapes hotels names and prices from Google
def get_hotels(location, date1, date2, number_of_adults):
    # If not valid, return so user can fix inputs
    if not check_valid_input(location, date1, date2):
        return
    
    try:
        hotels_text.delete("1.0", "end")
        
        # Headless option allows scraping without having to open the page in your browser
        # Source: https://www.zenrows.com/blog/headless-browser-python#switch-to-python-selenium-headless-mode
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        
        # Disabling images to make it run faster
        # Source: https://stackoverflow.com/questions/28070315/python-disable-images-in-selenium-google-chromedriver
        prefs = {"profile.managed_default_content_settings.images": 2}
        chrome_options.add_experimental_option("prefs", prefs)

        driver = webdriver.Chrome(options = chrome_options)

        driver.get("https://www.google.com/travel/")

        # Finding and clicking on the button that goes to the hotels page
        hotels_button = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/c-wiz[2]/div/div[2]/div/c-wiz/div/div/div[1]/div[2]/div/span[3]/a/button"))
        )

        hotels_button.click()

        # Finding the button to clear the location search bar 
        # and clicking it
        clear_search_bar = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/c-wiz[2]/div/c-wiz/div[1]/div[1]/div[1]/c-wiz/div/div/div[1]/div/div[1]/div[1]/div/div/div/div/div[1]/div/div/div[3]/button"))
        )
                                                    
        clear_search_bar.click()

        # Typing the location that was given by the user
        # into the search bar and hitting enter
        search_bar = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/c-wiz[2]/div/c-wiz/div[1]/div[1]/div[1]/c-wiz/div/div/div[1]/div/div[1]/div[1]/div/div/div/div/div[2]/div[2]/div/div[2]/input"))
        )
                                                    
        search_bar.send_keys(location) 
        search_bar.send_keys(Keys.ENTER)
        
        # Finding the box to type in the start date
        start_date = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/c-wiz[2]/div/c-wiz/div[1]/div[1]/div[1]/c-wiz/div/div/div[1]/div/div[1]/div[2]/div/div/div[2]/div[1]/div/input"))
        )
        
        # Deleting the default start date
        # and typing in the start date given by the user
        # and hitting enter
        start_date.send_keys(Keys.COMMAND + "a") 
        start_date.send_keys(Keys.DELETE)  
        start_date.send_keys(date1) 
        start_date.send_keys(Keys.ENTER)

        # Sleeping for one second so the start date does not revert back to the default
        time.sleep(1)

        # Finding the box to type in the end date
        end_date = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/c-wiz[2]/div/c-wiz/div[1]/div[1]/div[1]/c-wiz/div/div/div[1]/div/div[1]/div[2]/div/div/div[2]/div[2]/div/input"))
        )

        # Deleting the default end date
        # and typing in the end date given by the user
        # and hitting enter
        end_date.send_keys(Keys.COMMAND + "a") 
        end_date.send_keys(Keys.DELETE)  
        end_date.send_keys(date2) 
        end_date.send_keys(Keys.ENTER)
        
        # Clicking the arrow to show the options for number of people
        arrow = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/c-wiz[2]/div/c-wiz/div[1]/div[1]/div[1]/c-wiz/div/div/div[1]/div/div[1]/div[3]/div/div/div/div[1]/div"))
        )

        arrow.click()

        # Clicking the minus button to set number of adults to a default of one
        subtract_adult = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/c-wiz[2]/div/c-wiz/div[1]/div[1]/div[1]/c-wiz/div/div/div[1]/div/div[1]/div[3]/div/div/div/div[2]/div[2]/div[1]/div[1]/div[2]/div/span[1]/button"))
        )

        subtract_adult.click()

        add_adult = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/c-wiz[2]/div/c-wiz/div[1]/div[1]/div[1]/c-wiz/div/div/div[1]/div/div[1]/div[3]/div/div/div/div[2]/div[2]/div[1]/div[1]/div[2]/div/span[3]/button"))
        )

        # Loop that sets the number of adults to the user's chosen number
        for i in range(int(number_of_adults) - 1):
            add_adult.click()

        # Clicking the "Done" button to select the number of adults
        done = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/c-wiz[2]/div/c-wiz/div[1]/div[1]/div[1]/c-wiz/div/div/div[1]/div/div[1]/div[3]/div/div/div/div[2]/div[2]/div[2]/div[2]/button"))
        )

        done.click()

        # Sleeping for three seconds so it has time to scrape before the browser closes
        time.sleep(3)

        # Finding all sections containing info for each hotel 
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pjDrrc")))

        hotel_info = driver.find_elements(By.CSS_SELECTOR, ".pjDrrc")

        # Initializing a count so we can scrape 
        # the first ten results on the page
        count = 0
        
        # Loop that gets the names, prices, ratings, and hyperlinks
        for hotel in hotel_info:
            count += 1
            
            hotel_name = hotel.find_element(By.TAG_NAME, "h2")
            name = hotel_name.text
            name_text = f"{name}\n"

            # Prices with "Deal" next to them have a different selector
            # so we have a try except clause for each
            try:
                price_info = hotel.find_element(By.CSS_SELECTOR, "#id > c-wiz > c-wiz > div > div > div > div > div > div > div > div > a > div > div > div > span > span > span")
                price = price_info.text
                price_text = f"Price: {price}\n"
            except:
                try:
                    price_info = hotel.find_element(By.CSS_SELECTOR, "#id > c-wiz > c-wiz > div > div > div > div > div > div > div > div > a > div > div > div > div > span > span > span")
                    price = price_info.text
                    price_text = f"Price: Deal! {price}\n"
                # If no price is found, this message prints instead
                except:
                    price_text = f"Price not available\n"
                    
            try:
                rating_info = WebDriverWait(hotel, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "KFi5wf.lA0BZ")))
                rating = rating_info.text
                rating_text = f"Average rating: {rating}★\n"
            except:
                rating_text  = f"Rating not available\n"
                
            url_info = hotel.find_element(By.TAG_NAME, "a")
            url = url_info.get_attribute("href")
            
            # Adding this text into the hotels_text box
            hotels_text.insert(END, name_text)
            hotels_text.insert(END, price_text)
            hotels_text.insert(END, rating_text)
            hotels_text.insert(END, "Link to hotel page",hyperlink.add(partial(webbrowser.open,url)))
            hotels_text.insert(END, "\n------\n")
            
            # Breaking the loop once we've scraped 10 hotels
            if count == 10:
                break

    finally:
        driver.quit()

# Using messagebox widget to show error messages to the user if their input is invalid
# Source: https://docs.python.org/3/library/tkinter.messagebox.html
def check_valid_input(location, date1, date2):
    # Checking for empty inputs
    if location == "":
        messagebox.showerror("error", "Location box is empty.")
        return False
    elif date1 == "":
        messagebox.showerror("error", "Start date box is empty.")
        return False
    elif date2 == "":
        messagebox.showerror("error", "End date box is empty.")
        return False
    
    start_date = None
    end_date = None
    end_day = date2[8:10]
    end_month = date2[5:7]
    end_year = int(date2[:4])
        
    # Checking for invalid dates using datetime
    # Source: https://www.w3schools.com/python/python_datetime.asp 
    try:
        start_year = int(date1[:4])
        start_month = int(date1[5:7])
        start_day = int(date1[8:10])
        
        start_date = datetime.datetime(start_year, start_month, start_day)
    except:
        messagebox.showerror("error", "invalid start date")
        return False
    current_date = datetime.datetime.now()
    
    try:
        end_day = int(date2[8:10])
        end_month = int(date2[5:7])
        end_year = int(date2[:4])
        
        end_date = datetime.datetime(end_year, end_month, end_day)
    except:
        messagebox.showerror("error", "invalid end date")
        return False
    
    # Checking for dates in the past or too far into the future
    # Source: https://stackoverflow.com/questions/6871016/adding-days-to-a-date-in-python 
    current_date = datetime.datetime.now()
    if start_date < current_date:
        messagebox.showerror("error", "start date is before current date")
        return False
        
    if end_date < current_date:
        messagebox.showerror("error", "end date is before current date")
        return False
        
    if start_date > end_date:
        messagebox.showerror("error", "start date has to be before end date")
        return False
        
    if end_date > (current_date + datetime.timedelta(days = 300)):
        messagebox.showerror("error", "end date is too far into the future")
        return False
    
    return True
        
def clear_inputs():
    location.set("")
    date1.set("")
    date2.set("")
    weather_text.delete("1.0", "end")
    hotels_text.delete("1.0", "end")
    
# Source for adding hyperlinks to text widgets:
# https://www.tutorialspoint.com/how-to-create-hyperlink-in-a-tkinter-text-widget
def callback(url):
    webbrowser.open_new_tab(url)

# Frames
top_frame = LabelFrame(root, background = "MistyRose2")
top_frame.grid(row = 0, column = 0, padx = 5, pady = 4)

mid_frame = LabelFrame(root, background = "MistyRose2")
mid_frame.grid(row = 1, column = 0, padx = 5, pady = 4)

# Label and entry widget for location
location_label = Label(top_frame, text="Enter a location (city, state or city, country if outside the U.S.)")
location_entry = Entry(top_frame, width=35, borderwidth=5, textvariable=location)
location_label.grid(row = 0, column = 0, padx = 5, pady = 4)
location_entry.grid(row = 1, column = 0, padx = 5, pady = 4)

# Label and entry widget for start date (date1)
date1_label = Label(top_frame, text="Enter a start date (YYYY-MM-DD)")
date1_entry = Entry(top_frame, width=35, borderwidth=5, textvariable=date1)
date1_label.grid(row = 2, column = 0, padx = 5, pady = 4)
date1_entry.grid(row = 3, column = 0, padx = 5, pady = 4)

# Label and entry widgt for end date (date2)
date2_label = Label(top_frame, text="Enter an end date (YYYY-MM-DD)")
date2_entry = Entry(top_frame, width=35, borderwidth=5, textvariable=date2)
date2_label.grid(row = 4, column = 0, padx = 5, pady = 4)
date2_entry.grid(row = 5, column = 0, padx = 5, pady = 4)
    
# Initiating a string variable and setting it
# This variable is associated with the dropdown menu
number_of_adults = StringVar()
number_of_adults.set("Select number of adults")
    
# Creating the dropdown menu for number of adults
dropdown = OptionMenu(top_frame, number_of_adults, *number_list)
dropdown.grid(row = 6, column = 0, padx = 4, pady = 4)

# Text widget for weather
weather_text = Text(mid_frame, height=7, width = 50)
weather_text.grid(row = 0, column = 0, padx = 4, pady = 4)

# Button tied to the get_weather() function
weather_button = Button(mid_frame, font = 24, text = "Get Predicted Weather", 
                            command=lambda: get_weather(location.get(), date1.get(), date2.get()))
weather_button.grid(row = 1, column = 0, padx = 4, pady = 4)

# Text widget for hotels
hotels_text = Text(mid_frame, height = 10, width = 50, wrap=WORD)
v_scrollbar = Scrollbar(mid_frame, command = hotels_text.yview)
hotels_text.config(yscrollcommand = v_scrollbar.set)
hotels_text.grid(row = 2, column = 0, padx = 4, pady = 4)
v_scrollbar.grid(row = 2, column = 1, sticky = "nsew")

# Button tied to the get_hotels() function
# It takes a few second to load once you click the button
hotels_button = Button(mid_frame, font = 24, text = "Get Hotels", 
                            command=lambda: get_hotels(location.get(), date1.get(), date2.get(), number_of_adults.get()))
hotels_button.grid(row = 3, column = 0, padx = 4, pady = 4)

clear_button = Button(mid_frame, font = 24, text = "Clear Inputs", command=clear_inputs)
clear_button.grid(row = 4, column = 0, pady = 4)

# Source for HyperLinkManager snippet: 
# https://github.com/codewithdev/Code-Snippets/blob/master/tkinter/tkHyperlinkManager.py
hyperlink = HyperlinkManager(hotels_text)

root.mainloop()